In [ ]:
# Mount Google Drive Folder (/content/drive/My Drive/Colab)
from google.colab import drive
drive.mount('/content/drive') # 이 밑에있는 데이터 전부 mount 시키겠다

# import user defined files 
import sys
import_dir ="/content/drive/My Drive/Colab Notebooks/2. Classification/pytorch-cifar" 
sys.path.insert(0, import_dir) # 현재 경로 조정(다른파일 import할때 편하게 할 수 있음)

# # import ipynb files...
# !pip install import_ipynb 
# import import_ipynb

%run '/content/drive/My Drive/Colab Notebooks/2. Classification/pytorch-cifar/dataset.ipynb' # custom dataset 정의한 주피터파일 import

import easydict # argparse를 주피터에서 돌릴 수 있는 easydict


# Debugging Tool
import pdb

# Download & Unzip Dataset (Move from google drive to local)
%cp -r '/content/drive/My Drive/Colab Notebooks/Dataset/CIFAR10/cifar.tgz' '/content/sample_data' # 1에 있는 파일을 2에 넣는다.
!tar -xvf  '/content/sample_data/cifar.tgz' # tgz파일 압축해제 -> content 밑에서 풀림

# Initilize Additional parameters
train_root = "/content/cifar/train" # 데이터 경로를 담은 변수
test_root = "/content/cifar/test"

In [32]:
'''Train CIFAR10 with PyTorch.'''
# module import 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

from models import * # 폴더안에 모든걸 import하게 해줌
# models 안에있는 init.py 가 먼저 돌아간다.
# init.py에서 각각의 모델파일을 import 하고 있음 즉 init.py에 의해서 나머지 것들이 돌아감


from utils import progress_bar


'''
# argparse 라이브러리를 사용하기 원한다면 터미널이나 다른 프레임워크에서 실행해야함
# Removed Code (Because ipynb file does not support argparse)
# Even removed, these lines are substitued by easydict code

parser = argparse.ArgumentParser(description='PyTorch CIFAR10 Training')
parser.add_argument('--lr', default=0.1, type=float, help='learning rate')
parser.add_argument('--resume', '-r', action='store_true',
                    help='resume from checkpoint')
args = parser.parse_args()
'''
args = easydict.EasyDict({ "lr": 0.1, "resume": True})  # args.key값으로 접근 가능

device = 'cuda' if torch.cuda.is_available() else 'cpu' #  device설정-> cuda 사용가능하면 cuda사용 쓸수 없으면 CPU 사용

best_acc = 0  # best accuracy를 넣을 변수 초기화
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')

# 학습데이터에 쓰일 transform 정의
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4), # 32size로 랜덤하게 crop하고 left,right,top,bottom 테두리에 padding을 준다.
    transforms.RandomHorizontalFlip(), # 50% 확률로 좌우반전
    transforms.ToTensor(), # scaling, to tensor, channel dimension shift
    # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)), # 명도나 채도등 각각의 다른 이미지의 환경을 정규화하기위함 / 앞은 평균 뒤는 표준편차
])

transform_test = transforms.Compose([
    transforms.ToTensor(), # To tensor
    # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)), # Normalize
])

# dataset.ipynb에서 정의한 custom Dataset클래스 생성
trainset = Dataset(root=train_root, transforms=transform_train) 

# trainset을 Batch단위로 분할 + shuffle 등의 기능을 제공하는 DataLoader 생성
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=os.cpu_count(), drop_last=True) 

# num_workers=os.cpu_count() -> 데이터를 로딩하기위해 cpu개수만큼 사용한다.(data load multi processing)
# drop_last=True -> 마지막 데이터들이 batch_size로 나누어 떨어지지 않으면 버림 (batch_size가 128이면 80개 남음 -> 사용x)

# testset도 마찬가지
testset = Dataset(root=test_root, transforms=transform_test)

# testdata는 shuffle하지 않음
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=os.cpu_count(), drop_last=True)

# cifar10 class 종류
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')


# 사용할 모델 net으로 인스턴스화
print('==> Building model..')
# net = VGG('VGG19')
# net = ResNet18()
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
net = MobileNet() # 가장 가벼운 모델
# net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)
# net = EfficientNetB0()
# net = RegNetX_200MF()


net = net.to(device)  # 모델을 gpu로 올림

# gpu 있는거 다 쓰겠다.
if device == 'cuda':
    net = torch.nn.DataParallel(net) # 다수의 gpu로 병렬처리 (하지만 코랩은 gpu 1개)
    cudnn.benchmark = True 
'''
cudnn.benchmark = True ?

내장된 cudnn 자동 튜너를 활성화하여, 하드웨어에 맞게 사용할 최상의 알고리즘(텐서 크기나 conv 연산에 맞게?)을 찾는다.
입력 이미지 크기가 자주 변하지 않는다면, 초기 시간이 소요되지만 일반적으로 더 빠른 런타임의 효과를 볼 수 있다.
'''
 
# 현재 resume은 False
if args.resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')   
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'   # 모델 정보를 저장한 적이 없으면 에러
    checkpoint = torch.load('./checkpoint/ckpt.pth') # 저장되어있는 모델정보(weights, best_acc, epoch) 불러오기
    
    net.load_state_dict(checkpoint['net']) # 모델의 weight들을 불러온 weight값으로 업데이트
    best_acc = checkpoint['acc'] # 마찬가지로 best accuracy update
    start_epoch = checkpoint['epoch'] # 마찬가지

criterion = nn.CrossEntropyLoss() # multi class classification이므로 CrossEntropy 사용

# optimizer로 Stochastic Gradient Descent 사용 / learning rate는 0.1  / weight_decay는 L2 regularization
optimizer = optim.SGD(net.parameters(), lr=args.lr,
                      momentum=0.9, weight_decay=5e-4)

# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()  # train mode
    # loss, correct, total 값을 저장할 변수 초기화
    train_loss = 0  
    correct = 0
    total = 0    

    for batch_idx, (inputs, targets) in enumerate(trainloader): 
        # inputs.shape : (128, 3, 32, 32) / targets.shape : (128) / len(trainloader) : 390 

        inputs, targets = inputs.to(device), targets.to(device) # 데이터들을 gpu로 올림

        # forward
        optimizer.zero_grad() # optimizer 초기화


        outputs = net(inputs) # 모델에 inputs을 넣어서 outputs를 얻음  / outputs.shape : (128, 10)
        loss = criterion(outputs, targets)  # 128개의 image에 대한 L(w) 계산 

        # backward
        loss.backward() # 미분계산
        optimizer.step() # 파라미터 업데이트

        train_loss += loss.item() # 현재 batch의 L(w)를 train_loss에 계속 더함
        _, predicted = outputs.max(1) # 각 label의 예측값 중에서 가장 큰값의 인덱스를 뽑음 (128,10) -> (128)   
        total += targets.size(0) # 128씩 더함
        correct += predicted.eq(targets).sum().item() # 예측이 맞는것들에 대한것만 correct에 더함 (128개중 맞은개수)
        # pdb.set_trace()

        # 한 epoch에서의 정보를 출력하기 위해 utils.py에 있는 progress_bar 함수 사용
        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
        #print(batch_idx,'/', len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
        #             % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

# Validation
def test(epoch):
    global best_acc
    net.eval() # evaluation mode
    test_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            # batch_size : 100 / len(testloader) : 100
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            # no backward            
            test_loss += loss.item()
            _, predicted = outputs.max(1) # (100, 10) -> (100)
            total += targets.size(0) # 100씩 더함
            correct += predicted.eq(targets).sum().item() # 100개중 맞은개수만큼 더함

            progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
            #print(batch_idx,'/', len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            #             % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.   
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'): 
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc # best_acc 업데이트


for epoch in range(start_epoch, start_epoch+2):
    train(epoch)
    test(epoch)

==> Preparing data..
==> Building model..
==> Resuming from checkpoint..

Epoch: 61
 [================================================================>]  Step: 41ms | Tot: 25s371ms | Loss: 0.599 | Acc: 79.808% (39840/49920) 390/390 
 [================================================================>]  Step: 17ms | Tot: 2s989ms | Loss: 0.632 | Acc: 79.160% (7916/10000) 100/100 
Saving..

Epoch: 62
 [================================================================>]  Step: 43ms | Tot: 24s714ms | Loss: 0.601 | Acc: 79.537% (39705/49920) 390/390 
 [================================================================>]  Step: 20ms | Tot: 2s963ms | Loss: 0.816 | Acc: 72.920% (7292/10000) 100/100 


In [6]:
while 1:
    continue
    

KeyboardInterrupt: ignored